In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [3]:
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')

In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
train.columns.values

array(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], dtype=object)

In [7]:
#removing the unnecessary columns
train_df = train.drop(['Ticket', 'Cabin', 'Name', 'PassengerId'], axis = 1)
test_df = test.drop(['Ticket', 'Cabin', 'Name', 'PassengerId'], axis = 1)

In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Embarked    889 non-null object
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [9]:
#we have null values in "Age", so we fill those by Imputer

from sklearn.impute import SimpleImputer
num = train_df.select_dtypes(exclude=['object'])
my_imputer = SimpleImputer()
imputed_train = pd.DataFrame(my_imputer.fit_transform(num))
imputed_train.columns = num.columns
imputed_train

,Survived,Pclass,Age,SibSp,Parch,Fare
0,0.0,3.0,22.000000,1.0,0.0,7.2500
1,1.0,1.0,38.000000,1.0,0.0,71.2833
2,1.0,3.0,26.000000,0.0,0.0,7.9250
3,1.0,1.0,35.000000,1.0,0.0,53.1000
4,0.0,3.0,35.000000,0.0,0.0,8.0500
...,...,...,...,...,...,...
886,0.0,2.0,27.000000,0.0,0.0,13.0000
887,1.0,1.0,19.000000,0.0,0.0,30.0000
888,0.0,3.0,29.699118,1.0,2.0,23.4500
889,1.0,1.0,26.000000,0.0,0.0,30.0000


In [10]:
imputed_train.info()
#Alright, no null values now

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
Survived    891 non-null float64
Pclass      891 non-null float64
Age         891 non-null float64
SibSp       891 non-null float64
Parch       891 non-null float64
Fare        891 non-null float64
dtypes: float64(6)
memory usage: 41.9 KB


In [11]:
#Checking any null categorical Values
obj = train_df.select_dtypes(exclude = ['int64','float'])
obj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 2 columns):
Sex         891 non-null object
Embarked    889 non-null object
dtypes: object(2)
memory usage: 14.0+ KB


In [12]:
#fill it by max used value 
freq = obj.Embarked.dropna().mode()[0]
freq

'S'

In [13]:
obj['Embarked'] = obj['Embarked'].fillna(freq)
obj.info()
#No Null Values 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 2 columns):
Sex         891 non-null object
Embarked    891 non-null object
dtypes: object(2)
memory usage: 14.0+ KB


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
#Converting Categorical values to Numerical
from sklearn.preprocessing import OneHotEncoder
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_train = pd.DataFrame(OH_encoder.fit_transform(obj))
OH_train.index = obj.index
OH_train

,0,1,2,3,4
0,0.0,1.0,0.0,0.0,1.0
1,1.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...
886,0.0,1.0,0.0,0.0,1.0
887,1.0,0.0,0.0,0.0,1.0
888,1.0,0.0,0.0,0.0,1.0
889,0.0,1.0,1.0,0.0,0.0


In [15]:
train_fill = pd.concat([imputed_train, OH_train], axis = 1)
train_fill

,Survived,Pclass,Age,SibSp,Parch,Fare,0,1,2,3,4
0,0.0,3.0,22.000000,1.0,0.0,7.2500,0.0,1.0,0.0,0.0,1.0
1,1.0,1.0,38.000000,1.0,0.0,71.2833,1.0,0.0,1.0,0.0,0.0
2,1.0,3.0,26.000000,0.0,0.0,7.9250,1.0,0.0,0.0,0.0,1.0
3,1.0,1.0,35.000000,1.0,0.0,53.1000,1.0,0.0,0.0,0.0,1.0
4,0.0,3.0,35.000000,0.0,0.0,8.0500,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
886,0.0,2.0,27.000000,0.0,0.0,13.0000,0.0,1.0,0.0,0.0,1.0
887,1.0,1.0,19.000000,0.0,0.0,30.0000,1.0,0.0,0.0,0.0,1.0
888,0.0,3.0,29.699118,1.0,2.0,23.4500,1.0,0.0,0.0,0.0,1.0
889,1.0,1.0,26.000000,0.0,0.0,30.0000,0.0,1.0,1.0,0.0,0.0


In [16]:
train_fill.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
Survived    891 non-null float64
Pclass      891 non-null float64
Age         891 non-null float64
SibSp       891 non-null float64
Parch       891 non-null float64
Fare        891 non-null float64
0           891 non-null float64
1           891 non-null float64
2           891 non-null float64
3           891 non-null float64
4           891 non-null float64
dtypes: float64(11)
memory usage: 76.7 KB


In [17]:
#Applying all the above methods for test file
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Embarked    889 non-null object
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [18]:
#we have null values only in "Age" column
num_test = test_df.select_dtypes(exclude=['object'])
my_imputer = SimpleImputer()
imputed_test = pd.DataFrame(my_imputer.fit_transform(num_test))
imputed_test.columns = num_test.columns
imputed_test

,Pclass,Age,SibSp,Parch,Fare
0,3.0,34.50000,0.0,0.0,7.8292
1,3.0,47.00000,1.0,0.0,7.0000
2,2.0,62.00000,0.0,0.0,9.6875
3,3.0,27.00000,0.0,0.0,8.6625
4,3.0,22.00000,1.0,1.0,12.2875
...,...,...,...,...,...
413,3.0,30.27259,0.0,0.0,8.0500
414,1.0,39.00000,0.0,0.0,108.9000
415,3.0,38.50000,0.0,0.0,7.2500
416,3.0,30.27259,0.0,0.0,8.0500


In [19]:
obj_test = test_df.select_dtypes(exclude = ['int64','float'])
obj_test.info()
#No Null values = All Happies

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 2 columns):
Sex         418 non-null object
Embarked    418 non-null object
dtypes: object(2)
memory usage: 6.7+ KB


In [20]:
#Converting Categorical values to Numerical
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_test = pd.DataFrame(OH_encoder.fit_transform(obj_test))
OH_test.index = obj_test.index
OH_test

,0,1,2,3,4
0,0.0,1.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...
413,0.0,1.0,0.0,0.0,1.0
414,1.0,0.0,1.0,0.0,0.0
415,0.0,1.0,0.0,0.0,1.0
416,0.0,1.0,0.0,0.0,1.0


In [21]:
test_fill = pd.concat([imputed_test, OH_test], axis = 1)
test_fill

,Pclass,Age,SibSp,Parch,Fare,0,1,2,3,4
0,3.0,34.50000,0.0,0.0,7.8292,0.0,1.0,0.0,1.0,0.0
1,3.0,47.00000,1.0,0.0,7.0000,1.0,0.0,0.0,0.0,1.0
2,2.0,62.00000,0.0,0.0,9.6875,0.0,1.0,0.0,1.0,0.0
3,3.0,27.00000,0.0,0.0,8.6625,0.0,1.0,0.0,0.0,1.0
4,3.0,22.00000,1.0,1.0,12.2875,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
413,3.0,30.27259,0.0,0.0,8.0500,0.0,1.0,0.0,0.0,1.0
414,1.0,39.00000,0.0,0.0,108.9000,1.0,0.0,1.0,0.0,0.0
415,3.0,38.50000,0.0,0.0,7.2500,0.0,1.0,0.0,0.0,1.0
416,3.0,30.27259,0.0,0.0,8.0500,0.0,1.0,0.0,0.0,1.0


In [22]:
Y_train = train_fill['Survived']
Y_train

0      0.0
1      1.0
2      1.0
3      1.0
4      0.0
      ... 
886    0.0
887    1.0
888    0.0
889    1.0
890    0.0
Name: Survived, Length: 891, dtype: float64

In [23]:
X_train = train_fill.drop(['Survived'], axis = 1)
X_train

,Pclass,Age,SibSp,Parch,Fare,0,1,2,3,4
0,3.0,22.000000,1.0,0.0,7.2500,0.0,1.0,0.0,0.0,1.0
1,1.0,38.000000,1.0,0.0,71.2833,1.0,0.0,1.0,0.0,0.0
2,3.0,26.000000,0.0,0.0,7.9250,1.0,0.0,0.0,0.0,1.0
3,1.0,35.000000,1.0,0.0,53.1000,1.0,0.0,0.0,0.0,1.0
4,3.0,35.000000,0.0,0.0,8.0500,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
886,2.0,27.000000,0.0,0.0,13.0000,0.0,1.0,0.0,0.0,1.0
887,1.0,19.000000,0.0,0.0,30.0000,1.0,0.0,0.0,0.0,1.0
888,3.0,29.699118,1.0,2.0,23.4500,1.0,0.0,0.0,0.0,1.0
889,1.0,26.000000,0.0,0.0,30.0000,0.0,1.0,1.0,0.0,0.0


In [24]:
#Let us find the scores from different models and submit the best one
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(test_fill).astype(int)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

83.61

In [25]:
svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(test_fill)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

68.69

In [26]:
linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(test_fill)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
acc_linear_svc

/opt/conda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


74.07

In [27]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred_final = logreg.predict(test_fill).astype(int)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


80.25

In [28]:
gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(test_fill)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian

79.12

In [29]:
perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(test_fill)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

71.72

In [30]:
sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(test_fill)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd

77.78

In [31]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(test_fill)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

98.2

In [32]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(test_fill).astype(int)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

98.2

In [33]:
from xgboost import XGBClassifier 
my_model = XGBClassifier()
my_model.fit(X_train, Y_train)
predictions = my_model.predict(test_fill).astype(int)
acc_xgb = round(my_model.score(X_train, Y_train) * 100, 2)
acc_xgb

96.3

In [34]:
submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": Y_pred_final
    })

In [35]:
submission.to_csv('submission.csv', index=False)